In [8]:
# S3 prefix
prefix = "test-spacy-venkat-docker-example"

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [9]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [21]:
WORK_DIRECTORY = "spacy_data"

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

In [22]:
sess.default_bucket()

'sagemaker-us-west-1-849635460031'

In [16]:
account = sess.boto_session.client("sts").get_caller_identity()["Account"]
region = sess.boto_session.region_name
image = "849635460031.dkr.ecr.us-west-1.amazonaws.com/spacy-container:latest"

spacy = sage.estimator.Estimator(
    image,
    role,
    1,
    "ml.c4.2xlarge",
    output_path="s3://{}/output".format(sess.default_bucket()),
    sagemaker_session=sess,
)

spacy.fit(data_location)

INFO:sagemaker:Creating training-job with name: spacy-container-2023-05-23-22-20-15-175


2023-05-23 22:20:15 Starting - Starting the training job...
2023-05-23 22:20:29 Starting - Preparing the instances for training......
2023-05-23 22:21:31 Downloading - Downloading input data..Starting the training.
Created blank 'en' model
'ner' not in nlp.pipe_names
8
#015  0%|          | 0/8 [00:00<?, ?it/s]#015 50%|#####     | 4/8 [00:00<00:00, 32.53it/s]#015100%|##########| 8/8 [00:00<00:00, 34.99it/s]#015100%|##########| 8/8 [00:00<00:00, 34.56it/s]
{'ner': 184.22298562526703}
#015  0%|          | 0/8 [00:00<?, ?it/s]#015 50%|#####     | 4/8 [00:00<00:00, 34.02it/s]#015100%|##########| 8/8 [00:00<00:00, 36.93it/s]#015100%|##########| 8/8 [00:00<00:00, 36.43it/s]
{'ner': 62.24409560271624}
#015  0%|          | 0/8 [00:00<?, ?it/s]#015 50%|#####     | 4/8 [00:00<00:00, 37.02it/s]#015100%|##########| 8/8 [00:00<00:00, 35.85it/s]#015100%|##########| 8/8 [00:00<00:00, 35.99it/s]
{'ner': 23.862365433358548}
#015  0%|          | 0/8 [00:00<?, ?it/s]#015 50%|#####     | 4/8 [00:00<00:00, 

In [17]:
from sagemaker.predictor import csv_serializer

predictor = spacy.deploy(1, "ml.m4.xlarge", serializer=csv_serializer)

INFO:sagemaker:Creating model with name: spacy-container-2023-05-23-22-22-57-975
INFO:sagemaker:Creating endpoint-config with name spacy-container-2023-05-23-22-22-57-975
INFO:sagemaker:Creating endpoint with name spacy-container-2023-05-23-22-22-57-975


----!

In [18]:
print(predictor.predict("The London march came ahead of anti-war protests today in other cities , including Rome , Paris , and Madrid .").decode("utf-8"))

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


"('London', 'LOC')"
"('Rome', 'LOC')"
"('Paris', 'LOC')"
"('0', 'LOC')"



In [19]:
sess.delete_endpoint(predictor.endpoint)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Deleting endpoint with name: spacy-container-2023-05-23-22-22-57-975
